# online market simulator

## part1 : data preparation

In [1]:
from preparation.preparation import load_data,clean_data, summarize_data 

df = load_data("data.xlsx")
df_clean = clean_data(df)
summary = summarize_data(df_clean)
summary.head()
print(summary)

✅ Data loaded successfully! Shape: (525461, 8)
✅ Data cleaned! Remaining rows: 504731
✅ Summary created! 4495 unique products.
                           Description   AvgPrice  TotalQuantity  \
0                *boombox ipod classic  16.980000              1   
1             *usb office glitter lamp   8.650000              1   
2              *usb office mirror ball   8.650000              1   
3               10 colour spaceboy pen   0.980816           6142   
4       11 pc ceramic tea set polkadot   4.950000              3   
...                                ...        ...            ...   
4490      zinc heart lattice tray oval   5.971364            326   
4491       zinc metal heart decoration   1.387674          14120   
4492           zinc police box lantern   7.395000            195   
4493     zinc top  2 door wooden shelf  12.159091            237   
4494  zinc willie winkie  candle stick   1.085273           3950   

      UniqueCustomers  
0                   0  
1       

## part 2 : modeling sellers (you can see seller implementation in modeling package)

### example of sellers and their influencers network

In [2]:
from modeling.seller import Seller, create_sellers_for_product,get_multi_price_products
from random import randint
import pandas as pd

# get top multi price products
top_products = get_multi_price_products(df_clean,2)
# print(top_products)
# Create sellers
all_sellers: dict[str, list[Seller]] = {}

for product in top_products:
    sellers = create_sellers_for_product(df_clean, product)
    all_sellers[product] = sellers

for product, sellers in all_sellers.items():
    influence_scores = {s.name: randint(3, 8) for s in sellers}  # random influence
    for s in sellers:
        s.compute_demand(sellers, influence_scores[s.name])
        s.compute_profit()


results = []
for product, sellers in all_sellers.items():
    for s in sellers:
        results.append(s.info() | {"Product": product})

df_results = pd.DataFrame(results)


df_results


,Seller,Price,Ads,Demand,Profit,Product
0,10 colour spaceboy pen_Seller_1,0.72,8,62.14,18.84,10 colour spaceboy pen
1,10 colour spaceboy pen_Seller_2,0.85,14,66.36,23.29,10 colour spaceboy pen
2,10 colour spaceboy pen_Seller_3,1.66,12,58.50,68.26,10 colour spaceboy pen
3,12 ass zinc christmas decorations_Seller_1,2.10,15,74.12,78.39,12 ass zinc christmas decorations
4,12 ass zinc christmas decorations_Seller_2,4.21,15,57.46,178.64,12 ass zinc christmas decorations
5,12 ass zinc christmas decorations_Seller_3,4.30,6,53.42,178.83,12 ass zinc christmas decorations


## Step 3 — Apply simulation to all product groups

In [3]:
from market.simulation import simulate_nash

final_results = []

for product, sellers in all_sellers.items():
    print(f"Simulating market: {product}")
    final_sellers = simulate_nash(sellers,max_iterations=10000)
    
    for s in final_sellers:
        final_results.append(s.info() | {"Product": product})

df_final = pd.DataFrame(final_results)
df_final


Simulating market: 10 colour spaceboy pen
Simulating market: 12 ass zinc christmas decorations


,Seller,Price,Ads,Demand,Profit,Product
0,10 colour spaceboy pen_Seller_1,638.62,4993.0,2553.44,1624949.46,10 colour spaceboy pen
1,10 colour spaceboy pen_Seller_2,638.85,5000.0,2554.16,1626244.94,10 colour spaceboy pen
2,10 colour spaceboy pen_Seller_3,639.36,5006.0,2556.10,1628781.57,10 colour spaceboy pen
3,12 ass zinc christmas decorations_Seller_1,641.00,5008.0,2560.42,1634326.53,12 ass zinc christmas decorations
4,12 ass zinc christmas decorations_Seller_2,641.51,5015.0,2562.46,1636676.25,12 ass zinc christmas decorations
5,12 ass zinc christmas decorations_Seller_3,640.50,5006.0,2558.62,1631896.73,12 ass zinc christmas decorations


plot the simulation iteration

In [4]:
import matplotlib.pyplot as plt
plt.plot(history)
plt.xlabel("Iteration")
plt.ylabel("Total Market Profit")
plt.title("Market Convergence to Equilibrium")
plt.show()

NameError: name 'history' is not defined